<a href="https://colab.research.google.com/github/daegeunbae/daegeunbae/blob/main/Dacon_Project2022_04_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#인구 데이터 기반 소득 예측 경진대회
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import os       #운영체제 관련
import zipfile #압축 파일 처리 관련
from glob import glob
import sys

#시각화 패키지들
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("ggplot")

#분석 관련
from scipy.stats import chi2_contingency
from scipy.stats import kstest
from statsmodels.formula.api import ols

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = "/content/drive/MyDrive/02.coding/10.Team_Project/02.dacon_contest/train.csv"
train=pd.read_csv(path)
train.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country,target
0,0,32,Private,309513,Assoc-acdm,12,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
1,1,33,Private,205469,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,40,United-States,1
2,2,46,Private,149949,Some-college,10,Married-civ-spouse,Craft-repair,Husband,White,Male,0,0,40,United-States,0
3,3,23,Private,193090,Bachelors,13,Never-married,Adm-clerical,Own-child,White,Female,0,0,30,United-States,0
4,4,55,Private,60193,HS-grad,9,Divorced,Adm-clerical,Not-in-family,White,Female,0,0,40,United-States,0


In [ ]:
path1 = "/content/drive/MyDrive/02.coding/10.Team_Project/02.dacon_contest/test.csv"
test = pd.read_csv(path1)
test.head()

,id,age,workclass,fnlwgt,education,education.num,marital.status,occupation,relationship,race,sex,capital.gain,capital.loss,hours.per.week,native.country
0,0,47,Private,304133,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,45,United-States
1,1,34,Self-emp-inc,154227,Some-college,10,Never-married,Sales,Not-in-family,White,Male,0,0,75,United-States
2,2,31,Local-gov,158291,Bachelors,13,Never-married,Craft-repair,Not-in-family,White,Male,8614,0,40,United-States
3,3,28,Private,183155,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,0,55,United-States
4,4,54,Local-gov,182543,Some-college,10,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,Mexico


*데이터 수 확인

In [ ]:
train.info()
#3컬럼 정도가 na가 있음을 알 수있다.
#그리고 가설에서 예상했던 것처럼, workclass가 na면 ocupation도 na일 가능성이 높아보인다. (백수예상)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17480 entries, 0 to 17479
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              17480 non-null  int64 
 1   age             17480 non-null  int64 
 2   workclass       15644 non-null  object
 3   fnlwgt          17480 non-null  int64 
 4   education       17480 non-null  object
 5   education.num   17480 non-null  int64 
 6   marital.status  17480 non-null  object
 7   occupation      15637 non-null  object
 8   relationship    17480 non-null  object
 9   race            17480 non-null  object
 10  sex             17480 non-null  object
 11  capital.gain    17480 non-null  int64 
 12  capital.loss    17480 non-null  int64 
 13  hours.per.week  17480 non-null  int64 
 14  native.country  16897 non-null  object
 15  target          17480 non-null  int64 
dtypes: int64(8), object(8)
memory usage: 2.1+ MB


In [ ]:
train.describe()

,id,age,fnlwgt,education.num,capital.gain,capital.loss,hours.per.week,target
count,17480.000000,17480.000000,1.748000e+04,17480.000000,17480.000000,17480.00000,17480.000000,17480.000000
mean,8739.500000,38.720995,1.897610e+05,10.036556,1076.644508,83.87460,40.002460,0.234897
std,5046.185688,14.079617,1.049929e+05,2.604415,7439.498620,396.03288,12.671265,0.423947
min,0.000000,17.000000,1.228500e+04,1.000000,0.000000,0.00000,1.000000,0.000000
25%,4369.750000,27.000000,1.181558e+05,9.000000,0.000000,0.00000,38.000000,0.000000
50%,8739.500000,37.000000,1.781340e+05,10.000000,0.000000,0.00000,40.000000,0.000000
75%,13109.250000,48.000000,2.373180e+05,12.000000,0.000000,0.00000,45.000000,0.000000
max,17479.000000,90.000000,1.455435e+06,16.000000,99999.000000,4356.00000,99.000000,1.000000


In [ ]:
#컬럼명 변경 # . 이 있으면 분석시 오류가 생김 기존문자 대체
#train.columns = train.columns.str.replace('.','_')

In [ ]:
#test.columns = test.columns.str.replace('.','_')

In [ ]:
# 데이터 차원 확인
print('train : ',train.shape)
print("test : ", test.shape)

train :  (17480, 16)
test :  (15081, 15)


In [ ]:
np.sum(test.isna())

id                0
age               0
workclass         0
fnlwgt            0
education         0
education.num     0
marital.status    0
occupation        0
relationship      0
race              0
sex               0
capital.gain      0
capital.loss      0
hours.per.week    0
native.country    0
dtype: int64

In [ ]:
train = np.genfromtxt(path1, delimiter=",", skip_header=1).astype(np.dtype("uint8")) #genfromtxt 함수를 활용해 delimiter 쉼표로 구분 하겠다 

In [ ]:
import numpy as np

In [ ]:
train.info()

AttributeError: ignored

In [ ]:
len(test)

In [ ]:
train['workclass'].value_counts()

IndexError: ignored

In [ ]:
test_data.shape

In [ ]:
train_data.count()

In [ ]:
train_data.info()

In [ ]:
train_data['occupation'].value_counts()

In [ ]:
train_data['occupation']

In [ ]:
#Group by 적용
train_groupby = train_data.groupby(by='workclass')
print(type(train_groupby))
train_groupby

In [ ]:
# 결측치가 있는 row들을 확인
train[train.isna().sum(axis=1) > 0]
#workclass 와 occupation이 같이 nall값 존재

AttributeError: ignored

*이상치 확인(4개)

In [ ]:
train_data["workclass"].value_counts(normalize=True) #비율이나 갯수로

In [ ]:
train_data["occupation"].value_counts(normalize=True)

In [ ]:
train_data["native.country"].value_counts(normalize=True)

In [ ]:
train_data["target"].value_counts(normalize=True)

In [ ]:
# 빵구난 데이터들에 대한 체크 : fillna -> 함수화


In [ ]:
#구글에서 from sklearn.preprocessing검색하면 6.3.1, normalization, 구현할 시간에 잘 찾아서 써먹는게 중요!! MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
#적용할 모델 불러오기
min_max_sc = MinMaxScaler()
#주어진 데이터를 기반으로 룰을 잡고, 그 룰대로 변환!!:fit_train
train_norm = min_max_sc.fit_transform(train_data) #train_data를 기준으로 변형을 하겠다
#test_norm = min_max_sc.fit_transform(test_data)
test_norm = min_max_sc.transform(test_data)
### 주의사항!
#   -test는 정말로 모른다 생각하고 학습에서는 고려하면 안됨!
#   - train 만으로 규칙을 만들어서 train, test 모두 적용!

In [ ]:
# Base Line check
kfold = KFold(n_splits = 5, random_state=1234,shuffle=True )
rf = RandomForestClassifier(n_jobs=-1, random_state=1234)
scores = cross_val_score(rf, X_train, y_train ,
                         cv=kfold, scoring="accuracy")
for iter_count, acc in enumerate(scores):
    print("RF K-Fold {0}번째 검증 정확도 : {1:.4f}".format(iter_count,acc ))
print("RF K-Fold 평균 정확도 :", scores.mean())